In [1]:
import pandas as pd
import sqlite3
from datetime import datetime as dt
import hashlib
import sys
import os
import re
from pathlib import Path
file = Path(os.getcwd()).resolve()
parent = file.parent
sys.path.append(str(parent))
from importlib import reload
from workers import sql

In [2]:
dat=pd.read_csv("tab3.csv")
tab = "INDEXES"
db_file = '../trader.sqlite'

In [88]:
countries = sql.get_sql(tab="GEO",
                        get="country",
                        search=["%"],
                        cols=['country'],
                        db_file=db_file)['country']['country']
names = dat['name']

In [3]:
names = dat['name']

In [4]:
split = [re.split(' - ',n) for n in names]

In [5]:
name_short = [s[0] for s in split]
name_country = [s[1] for s in split]

In [22]:
a=pd.DataFrame([''])
a.loc[0,0]

''

In [8]:
[re.sub(r'INDEX','',n).strip() for n in name_short]

['ALL ORDINARIES',
 'HANG SENG',
 'JAKARTA COMPOSITE',
 'KUALA LUMPUR COMPOSITE',
 'KOSPI',
 'NIKKEI 225',
 'NZX 50',
 'PSEI',
 'SET',
 'SHANGHAI B-SHARE',
 'SHANGHAI COMPOSITE',
 'SENSEX 30',
 'STRAITS TIMES',
 'TAIEX']

In [90]:
#simplify countries - special cases
#name_country = [re.sub(r",.*$", "", c) for c in countries]
#name_country = [re.sub(r"\(.*$", "", c) for c in name_country]
name_country = [re.sub(r'SOUTH KOREA',r'KOREA, REP.',c) for c in name_country]

In [91]:
match = [re.search(c, r'-'.join(countries)) for c in name_country]

In [92]:
for i in range(len(match)):
    if not match[i]:
        name_short[i] = names[i]
        name_country[i] = ''

In [96]:
name_country

['AUSTRALIA',
 'HONG KONG',
 'INDONESIA',
 'MALAYSIA',
 'KOREA, REP.',
 'JAPAN',
 'NEW ZEALAND',
 'PHILIPPINES',
 'THAILAND',
 'CHINA',
 'CHINA',
 'INDIA',
 'SINGAPORE',
 'TAIWAN']

In [119]:
reload(sql)
resp=[sql.geo(get="iso2", search=[n+'%'], db_file=db_file) for n in name_country]

In [120]:
iso2=[]
for r in resp:
    if not r:
        iso2 +=['']
    else:
        iso2 +=[r['country']['iso2'].to_list()[0]]

In [121]:
iso2

['AU',
 'HK',
 'ID',
 'MY',
 'KR',
 'JP',
 'NZ',
 'PH',
 'TH',
 'CN',
 'CN',
 'IN',
 'SG',
 'TW']